<a href="https://colab.research.google.com/github/omid-sakaki-ghazvini/Machine-Learning/blob/main/Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
omidsakaki1370_data_preparation_example_path = kagglehub.dataset_download('omidsakaki1370/data-preparation-example')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.impute import KNNImputer

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# **Load Data**

In [ ]:
df = pd.read_csv('/kaggle/input/data-preparation-example/credit_risk_dataset.csv')

In [ ]:
df.info()

In [ ]:
missing_values_count = df.isnull().sum()
missing_values_count

# **Data Analysis**

In [ ]:
columns = {
    'loan_status': df['loan_status'],
    'loan_grade': df['loan_grade'],
    'cb_person_default_on_file': df['cb_person_default_on_file'],
    'person_home_ownership': df['person_home_ownership'],
    'loan_intent': df['loan_intent'],
}

def plot_data(name, data):
    plt.figure(figsize=(5, 5))
    palette_color = sns.color_palette('pastel')
    explode = [0.1 for _ in range(data.nunique())]

    target_counts = df.groupby(name)[name].count()

    target_counts.plot.pie(
    colors=palette_color,
    explode=explode,
    autopct="%1.1f%%",
    shadow=True,
    startangle=140,
    textprops={'fontsize': 14},
    wedgeprops={'edgecolor': 'black', 'linewidth': 1.5}
    )

    plt.title(name, fontsize=18, weight='bold')
    plt.axis('equal')
    plt.show()


for name, data in columns.items():
    plot_data(name, data)

# **step 1 : Handle Missing Values & Data Cleaning**

# [Data Cleaning Notebook](http://www.kaggle.com/code/omidsakaki1370/data-cleaning)
# [Missing Values Notebook](http://www.kaggle.com/code/omidsakaki1370/missing-values)

## **1-Normalization**

In [ ]:
columns = {'person_income','person_emp_length','loan_amnt','loan_int_rate','loan_percent_income','cb_person_cred_hist_length'}

def distplot_data(data):
    plt.figure(figsize=(8, 4))
    sns.distplot(data);

for i in columns:
    distplot_data(df[i])

In [ ]:
fig = plt.figure(figsize=(14, 4))
ax1 = plt.subplot(121)

sns.distplot(df['person_income']);
ax1.set_title("person_income");

df['person_income'] = np.log(df['person_income'])

ax1 = plt.subplot(122)
sns.distplot(df['person_income']);
ax1.set_title("person_income_log");

## **2-Outliers**

In [ ]:
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_out = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df_out

df = remove_outliers(df, 'person_age')
df = remove_outliers(df, 'person_income')
df = remove_outliers(df, 'person_emp_length')
df = remove_outliers(df, 'cb_person_cred_hist_length')
df.reset_index(drop=True, inplace=True)

## **3-Encoder**

In [ ]:
encoder=LabelEncoder()

df['person_home_ownership']=encoder.fit_transform(df['person_home_ownership'])
df['loan_intent']=encoder.fit_transform(df['loan_intent'])
df['loan_grade']=encoder.fit_transform(df['loan_grade'])
df['cb_person_default_on_file']=encoder.fit_transform(df['cb_person_default_on_file'])

## **4-missing values**

In [ ]:
missing_values_count = df.isnull().sum()
missing_values_count

In [ ]:
df = df.fillna(method = 'ffill')
missing_values_count = df.isnull().sum()

print(missing_values_count)
df.head(10)

In [ ]:
X = df.copy()
y = X.pop("loan_status")

# **step 2 : Feature Engineering**

# [Feature Engineering Notebook](https://www.kaggle.com/code/omidsakaki1370/feature-engineering)

In [ ]:
X_test = X.copy()

In [ ]:
from sklearn.feature_selection import VarianceThreshold

selector = VarianceThreshold()
X_new = selector.fit_transform(X_test, y)

print('X shape :', X.shape)
print('X_new shape :', X_new.shape)

## **Creating Features**

In [ ]:
X.head(10)

In [ ]:
X["New_Features"] = round(X.loan_int_rate - X.person_income).astype(int)

X[["loan_int_rate", "person_income", "New_Features"]].head(10)

In [ ]:
def F(x):
    if x>=0:
        return 1
    else:
        return 0

X['New_Features'] = X['New_Features'].apply(F)

print(X['New_Features'].head(10))
print(y.head(10))

## **Clustering With K-Means**

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4)

X["Cluster"] = kmeans.fit_predict(X)
X["Cluster"] = X["Cluster"].astype("category")
X.head()

In [ ]:
sns.relplot(x="person_income", y="loan_percent_income", hue="Cluster", data=X, height=4);

# **step 3 : Classification**

In [ ]:
#confusion matrix template
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):

    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

## **1-Split Data**

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

X=df.drop(columns = ['loan_status'],axis=1)
y=df['loan_status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

print("The size of the input train data is: {}".format(X_train.shape))
print("The size of the output train data is: {}".format(y_train.shape))
print("The size of the input test data is: {}".format(X_test.shape))
print("The size of the output test data is: {}".format(y_test.shape))

## **2-Scaler**

In [ ]:
scaler_data = StandardScaler()

X_train_scaled = scaler_data.fit_transform(X_train)
X_test_scaled = scaler_data.fit_transform(X_test)

fig = plt.figure(figsize=(10, 4))
sns.distplot(X_train_scaled);

## **3-Logistic Regression**

In [ ]:
# import the necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report,confusion_matrix

# LogisticRegression
LR = LogisticRegression()
LR.fit(X_train_scaled,y_train)

# Prediction
prediction=LR.predict(X_test_scaled)

#Plot Result
cm = confusion_matrix(prediction,y_test)
#print(cm)
plot_confusion_matrix(cm,
                          target_names=['1', '0'],
                          title='Logistic Regression',
                          cmap=None,
                          normalize=True)
sk_report = classification_report(
    digits=2,
    y_true=prediction,
    y_pred=y_test)
print(sk_report)

## **4-Decision Tree**

In [ ]:
# import the necessary libraries
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report,confusion_matrix

# Decision Tree
Tree = DecisionTreeClassifier()
Tree.fit(X_train_scaled,y_train)

# Prediction
prediction=Tree.predict(X_test_scaled)

#Plot Result
cm = confusion_matrix(prediction,y_test)
#print(cm)
plot_confusion_matrix(cm,
                          target_names=['1', '0'],
                          title='Decision Tree',
                          cmap=None,
                          normalize=True)
sk_report = classification_report(
    digits=2,
    y_true=prediction,
    y_pred=y_test)
print(sk_report)

## **5-Random Forest**

In [ ]:
# import the necessary libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report,confusion_matrix

# Random Forest
RF = RandomForestClassifier()
RF.fit(X_train_scaled,y_train)

# Prediction
prediction=RF.predict(X_test_scaled)

#Plot Result
cm = confusion_matrix(prediction,y_test)
#print(cm)
plot_confusion_matrix(cm,
                          target_names=['1', '0'],
                          title='Random Forest',
                          cmap=None,
                          normalize=True)
sk_report = classification_report(
    digits=2,
    y_true=prediction,
    y_pred=y_test)
print(sk_report)

## **6-Support Vector Machine**

In [ ]:
# import the necessary libraries
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report,confusion_matrix

# Support Vector Machine
SVM = SVC()
SVM.fit(X_train_scaled,y_train)

# Prediction
prediction=SVM.predict(X_test_scaled)

#Plot Result
cm = confusion_matrix(prediction,y_test)
#print(cm)
plot_confusion_matrix(cm,
                          target_names=['1', '0'],
                          title='Support Vector Machine',
                          cmap=None,
                          normalize=True)
sk_report = classification_report(
    digits=2,
    y_true=prediction,
    y_pred=y_test)
print(sk_report)

## **7-k-nearest neighbor**

In [ ]:
# import the necessary libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report,confusion_matrix

# k-nearest neighbor
KNN = KNeighborsClassifier(n_neighbors=15)
KNN.fit(X_train_scaled,y_train)

# Prediction
prediction=KNN.predict(X_test_scaled)

#Plot Result
cm = confusion_matrix(prediction,y_test)
#print(cm)
plot_confusion_matrix(cm,
                          target_names=['1', '0'],
                          title='k-nearest neighbor',
                          cmap=None,
                          normalize=True)
sk_report = classification_report(
    digits=2,
    y_true=prediction,
    y_pred=y_test)
print(sk_report)

## **8-Gradient Boosting**

In [ ]:
# import the necessary libraries
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report,confusion_matrix

# Gradient Boosting
GB = GradientBoostingClassifier( n_estimators=300,
                                 learning_rate=0.05,
                                 random_state=100,
                                 max_features=5)
GB.fit(X_train_scaled,y_train)

# Prediction
prediction=GB.predict(X_test_scaled)

#Plot Result
cm = confusion_matrix(prediction,y_test)
#print(cm)
plot_confusion_matrix(cm,
                          target_names=['1', '0'],
                          title='Gradient Boosting',
                          cmap=None,
                          normalize=True)
sk_report = classification_report(
    digits=2,
    y_true=prediction,
    y_pred=y_test)
print(sk_report)

## **9-MLP**

In [ ]:
from sklearn.neural_network import MLPClassifier

MLP = MLPClassifier(hidden_layer_sizes=(512), max_iter=50, solver='sgd', random_state=1, verbose=True)
MLP.fit(X_train_scaled, y_train)

# Prediction
prediction=MLP.predict(X_test_scaled)

#Plot Result
cm = confusion_matrix(prediction,y_test)
#print(cm)
plot_confusion_matrix(cm,
                          target_names=['1', '0'],
                          title='MLP',
                          cmap=None,
                          normalize=True)
sk_report = classification_report(
    digits=2,
    y_true=prediction,
    y_pred=y_test)
print(sk_report)

## **10-Voting Classifier**

In [ ]:
from sklearn.ensemble import VotingClassifier

Voting = VotingClassifier(
    estimators=[("Logistic Regression", LR), ("Decision Tree", Tree), ("Random Forest", RF),
               ("Support Vector Machine",SVM), ("k-nearest neighbor",KNN), ("Gradient Boosting",GB)],
    voting='hard',
)

Voting.fit(X_train_scaled,y_train)

# Prediction
prediction=Voting.predict(X_test_scaled)

#Plot Result
cm = confusion_matrix(prediction,y_test)
#print(cm)
plot_confusion_matrix(cm,
                          target_names=['1', '0'],
                          title='Voting Classifier',
                          cmap=None,
                          normalize=True)
sk_report = classification_report(
    digits=2,
    y_true=prediction,
    y_pred=y_test)
print(sk_report)

## **11-Cross-Validation**

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

mean_accuracy = []
i = 1
kf = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)

for train_index, test_index in kf.split(X_train_scaled, y_train):
    print('\n{} of kfold {}'.format(i, kf.n_splits))
    xtr, xvl = X.loc[train_index], X.loc[test_index]
    ytr, yvl = y[train_index], y[test_index]

    Voting.fit(xtr, ytr)
    pred_test = Voting.predict(xvl)
    score = accuracy_score(yvl, pred_test)
    mean_accuracy.append(score)
    print('accuracy_score', score)
    i+=1

print("\nMean validation accuracy: ", sum(mean_accuracy)/len(mean_accuracy))

In [ ]:
from statistics import stdev
from sklearn.model_selection import cross_val_score

score = cross_val_score(Voting, X_train_scaled, y_train, cv=5, scoring='recall')

Voting_cv_score = score.mean()
Voting_cv_stdev = stdev(score)

print('Cross Validation Recall scores are: {}'.format(score))
print('Average Cross Validation Recall score: ', Voting_cv_score)
print('Cross Validation Recall standard deviation: ', Voting_cv_stdev)

## **12-Pipelines**

In [ ]:
df = pd.read_csv('/kaggle/input/data-preparation-example/credit_risk_dataset.csv')
df.info()

In [ ]:
# Separate target from predictors
y = df.loan_status
X = df.drop(['loan_status'], axis=1)

# Divide data into train and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

print("The size of the input train data is: {}".format(X_train.shape))
print("The size of the output train data is: {}".format(y_train.shape))
print("The size of the input test data is: {}".format(X_test.shape))
print("The size of the output test data is: {}".format(y_test.shape))

In [ ]:
# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train.columns if X_train[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train[my_cols].copy()
X_test = X_test[my_cols].copy()

print("Categorical variables:")
print(categorical_cols)
print("numerical variables:")
print(numerical_cols)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant')),
    ('scaler', StandardScaler()),
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [ ]:
# Voting Classifier
model = VotingClassifier(
    estimators=[("Logistic Regression", LR), ("Decision Tree", Tree), ("Random Forest", RF),
               ("Support Vector Machine",SVM), ("k-nearest neighbor",KNN), ("Gradient Boosting",GB)],
    voting='hard',
)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
prediction = my_pipeline.predict(X_test)

#Plot Result
cm = confusion_matrix(prediction,y_test)
#print(cm)
plot_confusion_matrix(cm,
                          target_names=['1', '0'],
                          title='Random Forest',
                          cmap=None,
                          normalize=True)
sk_report = classification_report(
    digits=2,
    y_true=prediction,
    y_pred=y_test)
print(sk_report)